# Tutorial 1: Basics

In this tutorial we will exercise the basic functionaility of the `SQLAlchemy` package. We will:

* Create a simple `sqlite` database
* Create and load a table, with no regard to any relationships with other tables
* Insert data into the table
* Retrieve data from the table


### Requirements

You will need to have `sqlalchemy` installed:
```
pip install sqlalchemy
```

In [42]:
import sqlalchemy as sq
import os

## Creating a Database

The first step is to open a connection to a new `SQlite` database. If the database file already exists, a connection to it would be opened - but we want to start from scratch.

In [43]:
dbfile = 'basic.db'

try: os.remove(dbfile)
except: pass

In [44]:
db = sq.create_engine('sqlite:///'+dbfile)
db

Engine(sqlite:///basic.db)

Note that this does not actually create a database file yet:

In [45]:
!wc -c $dbfile

wc: basic.db: open: No such file or directory


You can mute the debug statements if you like:

In [46]:
db.echo = True  

## Creating a Table

Before creating tables, we need a `MetaData` object to manage them. Simply put, metadata keeps track of all table definitions (e.g., what the columns and their respective data types are). It also provides the connection to the database. 

In [47]:
# Instantiate a MetaData object:
metadata = sq.MetaData(db)

# Make a Table object, and give it the metadata object, and some new Column objects. 
# Note that this operation also updates the metadata, with the new column information.
object_table = sq.Table('Objects', metadata,
                        sq.Column('id', sq.Integer, primary_key=True),
                        sq.Column('ra', sq.Float),
                        sq.Column('dec', sq.Float))
object_table

Table('Objects', MetaData(bind=Engine(sqlite:///basic.db)), Column('id', Integer(), table=<Objects>, primary_key=True, nullable=False), Column('ra', Float(), table=<Objects>), Column('dec', Float(), table=<Objects>), schema=None)

In [48]:
metadata.__dict__

{'_bind': Engine(sqlite:///basic.db),
 '_fk_memos': defaultdict(list, {}),
 '_schemas': set(),
 '_sequences': {},
 'naming_convention': immutabledict({'ix': 'ix_%(column_0_label)s'}),
 'schema': None,
 'tables': immutabledict({'Objects': Table('Objects', MetaData(bind=Engine(sqlite:///basic.db)), Column('id', Integer(), table=<Objects>, primary_key=True, nullable=False), Column('ra', Float(), table=<Objects>), Column('dec', Float(), table=<Objects>), schema=None)})}

Now that we have a `Table` instance, we need to have it ingested by the database. This is done with the `create` method:

In [49]:
object_table.create()

2016-08-30 21:44:06,180 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2016-08-30 21:44:06,183 INFO sqlalchemy.engine.base.Engine ()
2016-08-30 21:44:06,185 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2016-08-30 21:44:06,186 INFO sqlalchemy.engine.base.Engine ()
2016-08-30 21:44:06,189 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Objects" (
	id INTEGER NOT NULL, 
	ra FLOAT, 
	dec FLOAT, 
	PRIMARY KEY (id)
)


2016-08-30 21:44:06,190 INFO sqlalchemy.engine.base.Engine ()
2016-08-30 21:44:06,194 INFO sqlalchemy.engine.base.Engine COMMIT


In [50]:
!wc -c $dbfile

    8192 basic.db


See how `sqlalchemy` wrote some SQL for you, and then executed it? :-)

## Inserting Data

Now we have a database, we can add more data to it - either more rows ("records") or columns ("attributes").

If a table is already defined (which it is now), we can simply load it into a `Table` object, via the `metadata` object:

In [52]:
data = sq.Table('Objects', metadata, autoload=True)
data

Table('Objects', MetaData(bind=Engine(sqlite:///basic.db)), Column('id', Integer(), table=<Objects>, primary_key=True, nullable=False), Column('ra', Float(), table=<Objects>), Column('dec', Float(), table=<Objects>), schema=None)

To insert data, we first ask for an insert statement that references to astro_objects data table. Afterwards, we use execute() method to insert some data rows. 

Notice that in SQLAlchemy, you do not explicitly provide SQL statements, you just work with python objects... 

In [53]:
db.echo = False

row = {'ra': 10.23, 'dec': -32.34}

inserter = data.insert()
for k in range(1000): 
    inserter.execute(row)

In [54]:
!wc -c $dbfile

   36864 basic.db


## Retrieving Data

We can fetch all entries by using `select()` method. Calling `fetchall()` returns all data. This is similar to the following SQL statement.

    SELECT * FROM Objects

In [55]:
x = object_table.select().execute().fetchall()
type(x), len(x)

(list, 1000)

In [56]:
print x[0]

(1, 10.23, -32.34)


In the next tutorial, we will see how to retrieve larger amounts of data efficiently, using `Session` objects.

For now though, let's do one last demo. We can start a second connection to the same database file, and read data from it, as follows:

In [58]:
db2 = sq.create_engine('sqlite:///'+dbfile)
metadata2 = sq.MetaData(db)
data2 = sq.Table('Objects', metadata2, autoload=True)
x2 = data2.select().execute().fetchall()
x2

[(1, 10.23, -32.34),
 (2, 10.23, -32.34),
 (3, 10.23, -32.34),
 (4, 10.23, -32.34),
 (5, 10.23, -32.34),
 (6, 10.23, -32.34),
 (7, 10.23, -32.34),
 (8, 10.23, -32.34),
 (9, 10.23, -32.34),
 (10, 10.23, -32.34),
 (11, 10.23, -32.34),
 (12, 10.23, -32.34),
 (13, 10.23, -32.34),
 (14, 10.23, -32.34),
 (15, 10.23, -32.34),
 (16, 10.23, -32.34),
 (17, 10.23, -32.34),
 (18, 10.23, -32.34),
 (19, 10.23, -32.34),
 (20, 10.23, -32.34),
 (21, 10.23, -32.34),
 (22, 10.23, -32.34),
 (23, 10.23, -32.34),
 (24, 10.23, -32.34),
 (25, 10.23, -32.34),
 (26, 10.23, -32.34),
 (27, 10.23, -32.34),
 (28, 10.23, -32.34),
 (29, 10.23, -32.34),
 (30, 10.23, -32.34),
 (31, 10.23, -32.34),
 (32, 10.23, -32.34),
 (33, 10.23, -32.34),
 (34, 10.23, -32.34),
 (35, 10.23, -32.34),
 (36, 10.23, -32.34),
 (37, 10.23, -32.34),
 (38, 10.23, -32.34),
 (39, 10.23, -32.34),
 (40, 10.23, -32.34),
 (41, 10.23, -32.34),
 (42, 10.23, -32.34),
 (43, 10.23, -32.34),
 (44, 10.23, -32.34),
 (45, 10.23, -32.34),
 (46, 10.23, -32.34